<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 IMDB دسته بندی نظرات
</font>
</h1>



<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این تمرین، قصد داریم بر روی یک مجموعه داده نظرات فیلم، طبقه‌بندی احساسات انجام دهیم. ما قصد داریم یک شبکه Feedforward Net، یک شبکه Recurrent Net ساخته و یکی یا چند تا از آن‌ها را ترکیب کنیم تا عملکرد هر یک از آن‌ها را درک کنیم. همانطور که می‌دانید یک جمله می‌تواند به عنوان توالی از کلماتی در نظر گرفته شود که در کنار یکدیگر معنایی را میرسانند. هر کدام از کلمات بر معنای جمله تأثیر می‌گذارند. کلماتی که در ابتدای جمله ظاهر می‌شوند، ساختار و معنای جمله را در قسمت‌های بعدی جمله تحت تأثیر قرار می‌دهند . به همین ترتیب، کلماتی که در اواخر جمله ظاهر می‌شوند، می‌توانند بر معنای کلمات ابتدایی تأثیر بگذارند.

In [ ]:
# !pip install --upgrade keras

In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import LSTM, SimpleRNN
# from keras.layers.embeddings import Embedding
from keras.layers import Embedding

from keras.layers import Flatten
from keras.preprocessing import sequence
from keras.layers import Conv1D
from keras.layers import MaxPool1D
# from keras.layers.embeddings import Embedding
import numpy as np
# fix random seed for reproducibility
numpy.random.seed(1)

In [2]:
# We want to have a finite vocabulary to make sure that our word matrices are not arbitrarily small
vocabulary_size = 10000

# We also want to have a finite length of reviews and not have to process really long sentences.
max_review_length = 500


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
همان طور که میدانیم یک توکن به یک واحد معنایی منفرد و اتمی (یعنی یک کلمه) اشاره دارد.هر کلمه متمایز را به یک عدد متمایز (عدد صحیح)تبدیل می شود.
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
خوشبختانه، مجموعه داده ما از قبل به چنین شکلی توکن‌سازی شده است.در نتیجه صرفا نیاز است دیتا را لود کنید.




In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print('Number of reviews', len(X_train))
print('Length of first and fifth review before padding', len(X_train[0]) ,len(X_train[4]))
print('First review', X_train[0])
print('First label', y_train[0])

17464789/17464789 [==============================] - 0s 0us/step
Number of reviews 25000
Length of first and fifth review before padding 218 147
First review [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104,

In [4]:
X_train.shape

(25000,)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
از آن جایی که طول جملات در یک کتن با یکدیگر متفاوت هستند،و ما جملات را به صورت دسته ای به مدل میدهیم نیاز داریم که با استفاده از padding طول جملات را یکسان نماییم.

In [ ]:
# from keras.preprocessing.sequence import pad_sequences

In [5]:
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print('Length of first and fifth review after padding', len(X_train[0]) ,len(X_train[4]))

Length of first and fifth review after padding 500 500


In [6]:
X_train.shape

(25000, 500)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
اکنون مدل اول خود را بسازید. این مدل ک مدل شبکه عصبی معمولی است با یک لایه پنهان که 250 نود  دارد.در مورد تابع فعال ساز یا تابع بهینه ساز شما آزاد هستید که از هر کدام که میخواهید استفاده نمایید.



In [7]:
from keras import layers

In [11]:
model1 = Sequential()
model1.add(layers.Dense(250,activation='relu',input_shape=(500,)))
model1.add(Dense(1))
model1.add(Activation("sigmoid"))
 #using sigmoid for binary classification
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 250)               125250    
                                                                 
 dense_5 (Dense)             (None, 1)                 251       
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 125501 (490.24 KB)
Trainable params: 125501 (490.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128, verbose=2)

Epoch 1/10
196/196 - 4s - loss: 179.8795 - accuracy: 0.5064 - val_loss: 97.1179 - val_accuracy: 0.5040 - 4s/epoch - 18ms/step
Epoch 2/10
196/196 - 1s - loss: 50.2837 - accuracy: 0.5911 - val_loss: 54.4673 - val_accuracy: 0.5024 - 1s/epoch - 7ms/step
Epoch 3/10
196/196 - 1s - loss: 18.6582 - accuracy: 0.6673 - val_loss: 34.8628 - val_accuracy: 0.5100 - 1s/epoch - 7ms/step
Epoch 4/10
196/196 - 1s - loss: 8.2401 - accuracy: 0.7268 - val_loss: 24.3985 - val_accuracy: 0.5047 - 1s/epoch - 8ms/step
Epoch 5/10
196/196 - 1s - loss: 4.0816 - accuracy: 0.7723 - val_loss: 17.0369 - val_accuracy: 0.5092 - 1s/epoch - 6ms/step
Epoch 6/10
196/196 - 1s - loss: 2.2936 - accuracy: 0.8027 - val_loss: 13.2722 - val_accuracy: 0.5098 - 1s/epoch - 6ms/step
Epoch 7/10
196/196 - 1s - loss: 1.5301 - accuracy: 0.8183 - val_loss: 10.7293 - val_accuracy: 0.5090 - 1s/epoch - 6ms/step
Epoch 8/10
196/196 - 1s - loss: 1.0702 - accuracy: 0.8358 - val_loss: 9.3616 - val_accuracy: 0.5079 - 1s/epoch - 7ms/step
Epoch 9/10
1

In [16]:
scores1 = model1.evaluate(X_test, y_test, verbose=2)
print("Accuracy: %.2f" %(scores1[1]*100))

782/782 - 1s - loss: 8.9603 - accuracy: 0.5079 - 1s/epoch - 1ms/step
Accuracy: 50.79


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
همان طور که مشاهده کردید نتیجه مدل چندان جالب نبود. اکنون از شما می‌خواهم که از
embedding  استفاده کنید و بنویسید چرا استفاده از embeding موجب افزایش کارایی مدل می‌شود.

In [27]:
maxlen=len(X_train[0])
print(maxlen)

500


In [28]:
nb_words=vocabulary_size
#10000
# nb_words=max_review_length
 #500

In [37]:
embedding_dims = 100
model2 = Sequential()

model2.add(Embedding(nb_words,embedding_dims,input_length=maxlen))
model2.add(layers.Dense(250,activation='relu'))
model2.add(Dense(1))
# model2.add(Activation("sigmoid"))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
print(model2.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dense_19 (Dense)            (None, 500, 250)          25250     
                                                                 
 dense_20 (Dense)            (None, 500, 1)            251       
                                                                 
Total params: 1025501 (3.91 MB)
Trainable params: 1025501 (3.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [39]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 89s 227ms/step - loss: 0.7441 - accuracy: 0.5039 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 2/3
391/391 [==============================] - 88s 224ms/step - loss: 0.6940 - accuracy: 0.4966 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 3/3
391/391 [==============================] - 86s 219ms/step - loss: 0.6942 - accuracy: 0.4980 - val_loss: 0.6933 - val_accuracy: 0.5001


In [40]:
scores2 = model2.evaluate(X_test, y_test, verbose=2)

782/782 - 18s - loss: 0.6933 - accuracy: 0.5002 - 18s/epoch - 23ms/step


In [41]:
print("Accuracy: %.2f" %(scores2[1]*100))

Accuracy: 50.02


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
اکنون مدل سوم خود را با embedding  و یک simple rnn  بسازید

In [42]:
rnn = Sequential()

rnn.add(Embedding(nb_words,embedding_dims,input_length =len(X_train[0]))) # num_words=15000
rnn.add(SimpleRNN(16,input_shape = (nb_words,maxlen), return_sequences=False,activation="relu"))
rnn.add(Dense(1))
rnn.add(Activation("sigmoid"))

print(rnn.summary())
rnn.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 16)                1872      
                                                                 
 dense_21 (Dense)            (None, 1)                 17        
                                                                 
 activation_7 (Activation)   (None, 1)                 0         
                                                                 
Total params: 1001889 (3.82 MB)
Trainable params: 1001889 (3.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [43]:
rnn.fit(X_train, y_train, epochs=20, batch_size=16)

Epoch 1/20
1563/1563 [==============================] - 180s 114ms/step - loss: 0.5050 - accuracy: 0.7497
Epoch 2/20
1563/1563 [==============================] - 172s 110ms/step - loss: 0.3216 - accuracy: 0.8693
Epoch 3/20
1563/1563 [==============================] - 175s 112ms/step - loss: 0.2672 - accuracy: 0.8979
Epoch 4/20
1563/1563 [==============================] - 174s 111ms/step - loss: 0.2421 - accuracy: 0.9109
Epoch 5/20
1563/1563 [==============================] - 172s 110ms/step - loss: 0.2252 - accuracy: 0.9212
Epoch 6/20
1563/1563 [==============================] - 173s 111ms/step - loss: 0.2073 - accuracy: 0.9278
Epoch 7/20
1563/1563 [==============================] - 171s 109ms/step - loss: 0.1908 - accuracy: 0.9291
Epoch 8/20
1563/1563 [==============================] - 177s 113ms/step - loss: 0.1778 - accuracy: 0.9380
Epoch 9/20
1563/1563 [==============================] - 173s 111ms/step - loss: 0.1734 - accuracy: 0.9437
Epoch 10/20
1563/1563 [=======================

In [44]:
scores_rnn = rnn.evaluate(X_test, y_test, verbose=2)
print("Accuracy: %.2f" %(scores_rnn[1]*100))

782/782 - 20s - loss: 0.4893 - accuracy: 0.8411 - 20s/epoch - 25ms/step
Accuracy: 84.11


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
و مدل آخر را نیز با یک lstm بسازید و این 4 مدل را با یک دیگر مقایسه نمایید.


In [50]:
look_back = 1

In [57]:
rnn_lstm = Sequential()
rnn_lstm.add(Embedding(nb_words,embedding_dims,input_length=maxlen))
rnn_lstm.add(LSTM(4, input_shape=(1, look_back)))
rnn_lstm.add(Dense(1))

In [58]:
rnn_lstm.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [59]:
print(rnn_lstm.summary())


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 500, 100)          1000000   
                                                                 
 lstm_3 (LSTM)               (None, 4)                 1680      
                                                                 
 dense_25 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1001685 (3.82 MB)
Trainable params: 1001685 (3.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [61]:
rnn_lstm.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 72s 183ms/step - loss: 0.1857 - accuracy: 0.9817
Epoch 2/3
391/391 [==============================] - 71s 183ms/step - loss: 0.1686 - accuracy: 0.9858
Epoch 3/3
391/391 [==============================] - 70s 180ms/step - loss: 0.1676 - accuracy: 0.9868


In [62]:
scores_rnn_lstm = rnn_lstm.evaluate(X_test, y_test, verbose=2)
print("Accuracy: %.2f" %(scores_rnn_lstm[1]*100))

782/782 - 24s - loss: 1.3156 - accuracy: 0.8608 - 24s/epoch - 30ms/step
Accuracy: 86.08


LSTM همون نسخه بهتر شده شبکه های عصبی بازگشتی هست و بخاطر حافظه ی بیشتری که داره، میتونه اطلاعات بیشتری رو نگهداری کنه و برای داده های متوالی، کارایی بیشتری از خودش نشون میده نسبت به شبکه های عصبی ساده.
حافظه هایی که در LSTM هستن، به سه نوع تقسیم میشن.
گیت ورودی، گیتی که اطلاعاتش فراموش میشن و گیت برای خروجی ها و این ها مسئولیت کنترل حافظه توی معموری هر سلول رو دارن.
با توجه به پیچیدگی بیشتر LSTM نسبت به شبکه های عصبی ساده، انتظار میره که دقتش کمتر باشه و مقدار خطا بیشتر بشه.

ساختار LSTM شامل 4 لایه شبکه عصبی هست که که هر کدوم با هم تفاوت دارن.
این درحالیه که RNN به تنهایی قادره تا بخش کوچکی از اطلاعات رو در خودش ذخیره کنه و نهایتا تا 30 ثانیه میتونه این اطلاعات رو به یاد بیاره.

در مرحله دوم که از EMBEDDEING استفاده کردیم، ورودی های توالی به ابعاد بزرگتر تغییر میکنن و پیچیدگی مدل کاهش پیدا میکنه و به تبع اون، دقت هم کمتر میشه.

در مدل ابتدایی هم که ساختیم، پیچیدگی به شدت کم هست و بین داده های ورودی، ارتباط پیچیده ای وجود نداره. به همین خاطر، دقت پایین هست.